In [3]:
import shap
from final_utils import *
from keras.layers import Concatenate
from MultiGap_CNN_FC import *

In [4]:
model_base = model_inceptionresnet_multigap()
model_cnn = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1",trainable=False) ])   
model_fc = fc_model_softmax(input_num=9744)
model_fc.load_weights("models/Softmax/MG_CNN/model_fc_softmax_MG_8k_B7_1k_600x600.hdf5")#model_fc_softmax_MG_8k_B7_1k_600x600

In [5]:
pca_mg_path = 'models/PCA/PCA_MG_8464_auto.pkl'
pca_cnn_path= 'models/PCA/PCA_CNN_1280_auto.pkl'
pca_mg = pk.load(open(pca_mg_path,'rb'))
pca_cnn = pk.load(open(pca_cnn_path,'rb'))


## Creating model

In [6]:
model =  model_multigap_cnn_fc( pca_mg = True, pca_cnn = True)

In [7]:
root_path = generate_root_path()
good_imgs_path = glob(os.path.join(root_path, 'Data', 'benchmark', 'images', 'good', '*'))
bad_imgs_path = glob(os.path.join(root_path, 'Data', 'benchmark', 'images', 'bad', '*'))
paths_bench = good_imgs_path + bad_imgs_path
labels_bench = np.concatenate((np.ones(len(good_imgs_path)) ,np.zeros(len(bad_imgs_path))))

## Some useful functions

In [8]:
def calc_acc(labels, predicted):
    return np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1)) / len(labels)

def predicts(x, model):
    try:
        pred = model.predict(x, verbose=0)
    except:
        x = x[None] #changed 02.08 for evaluator visualizing predictions
        pred = model.predict(x, verbose=0)
   
    return pred

img = read_img(path=paths_bench[0], resize_func=None, size=None, for_all=False)
img = np.squeeze(img, axis=0)
print(img.shape)

def predict_from_path_false_pred(model, paths, model_CNN=None, resize_func=None, size=None, for_all=False, save_results=None, save_to=None, 
                     is_norm=True):
    #always requires list of paths
    predicted = []
    false_pred = []
    
    for path in paths:
        
        if 'good' in path:
            label = 1
        elif 'bad' in path:
            label = 0

        img = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all)
        img = np.squeeze(img, axis=0)
        print(img.shape)
        
        print(model)
        pred_score = predicts(img, model)
    
        if np.argmax(pred_score) != label:
            false_pred.append(path)
            
        predicted.append(pred_score)
    
    predicted = np.array(predicted)
    predicted = np.squeeze(predicted)
    
    if save_results:
        np.save(save_to, np.argmax(predicted, axis=-1))
        
    return predicted, false_pred

# def predict_from_path(model_gap, model, paths, resize_func=None, size=None, for_all=False, save_results=None,
#                       save_to=None, model_CNN=None, is_norm=False):
#     #always requires list of paths
#     predicted = []
#     false_pred = []
    
#     for path in paths:
#         if 'good' in path:
#             label = 1
#         elif 'bad' in path:
#             label = 0
    
# #     for i, path in enumerate(paths):
#         img_mg = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all)
#         img_cnn = None
#         if model_CNN:
#             img_cnn = read_img(path=path, resize_func=resize_add_border, size=(600, 600))
#         pred_score = predict(img_mg, img_cnn, model_gap, model, model_CNN, is_norm)
#         if np.argmax(pred_score) != label:
#             false_pred.append(path)
            
#         predicted.append(pred_score)
    
#     predicted = np.array(predicted)
#     predicted = np.squeeze(predicted)
    
#     if save_results:
#         np.save(save_to, np.argmax(predicted, axis=-1))
        
#     return predicted, false_pred

(875, 1200, 3)


## Saparating false predicted images


In [5]:
predicted, false_pred = predict_from_path_false_pred(model, paths_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_cnn, is_norm=False)# 

In [ ]:
predicted

In [10]:
# url = f'{root_path}Data/benchmark/images/bad/2022-02-20-11-24-fortress-7024370_1280.jpg'
# url = false_pred_1[0]

def read_and_transform_img(url):
    img = Image.open(url)
    img = img_to_array(img)
#     img = img / 255
#     img = resize_max(img, size=(996,996))
    print(img.shape)
    img = np.expand_dims(img, axis=0)

    return img



In [7]:
# These are the paths for the trained FC, WITH normalization during validation
acc = calc_acc(labels_bench, predicted1)
print(f'Accuracy: {acc}')
for path in false_pred_1:
    print(path)
len(false_pred_1)

In [ ]:
path = "XAI/gal-angebot-portrait-1.jpg"

images = read_and_transform_img(path)
print(type(images))
preds = model.predict(images)
prediction = np.argmax(preds)
pct = np.max(preds)
if prediction == 0:
    print('It\'s no aesthethic!')
elif prediction == 1:
    print('It\'s an aesthethic!')
print(preds)

(1079, 894, 3)
<class 'numpy.ndarray'>
image shape (None, 1079, 894, 3)
Resize max (1079, 894, 3)
image shape after resize max (1, 996, 825, 3)
mg feature vector shape (1, 16928)
image shape after resize border (1, 600, 600, 3)
cnn feature vector shape (None, 2560)
mg feature vector shape after pca (1, 8464)
cnn feature vector shape after pca (None, 1280)
feature vector shape after concat (1, 9744)
feature vector shape after fc (1, 2)
print before return


In [7]:
images.shape

(1, 1079, 894, 3)

## Explanation for false predictions with SHAP


In [ ]:
masker = shap.maskers.Image("blur"+ str(images.shape[1:3]), images.shape[1:]) #"blur"+ str(images.shape[1:3])

In [ ]:
explainer = shap.Explainer(model, masker, output_names=[0, 1])

In [ ]:
shap_values = explainer(images, max_evals = 1000, batch_size = 1, outputs = shap.Explanation.argsort.flip[:2])
shap.image_plot(shap_values)
